In [ ]:
! pip install mlflow awscli boto3 optuna imbalanced-learn lightgbm

In [ ]:
! aws configure

In [3]:
import mlflow
mlflow.set_tracking_uri('http://ec2-100-26-36-125.compute-1.amazonaws.com:5000/')

In [4]:
mlflow.set_experiment('E LightGBM HP Tuning')

<Experiment: artifact_location='s3://om-mlflow-bucket/621100004064027216', creation_time=1741370223256, experiment_id='621100004064027216', last_update_time=1741370223256, lifecycle_stage='active', name='E LightGBM HP Tuning', tags={}>

In [5]:
import pandas as pd
df=pd.read_csv('/content/reddit_processed.csv').dropna()

In [6]:
import pandas as pd
import mlflow
import mlflow.sklearn
import optuna
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,classification_report
from imblearn.over_sampling import SMOTE
from lightgbm import LGBMClassifier
import matplotlib.pyplot as plt



In [7]:
# Remap the Class Labels
df['category']=df['category'].map({-1:2,0:0,1:1})

df=df.dropna(subset=['category'])

In [8]:
# Train Test Split
X=df['clean_comment']
y=df['category']
X_train,X_test,y_train,y_test=train_test_split(X ,y,test_size=0.2,random_state=42,stratify=y)

In [9]:
# TfidfVectorizer Setup
ngram_range=(1,3)
max_features=10000

vectorizer=TfidfVectorizer(ngram_range=ngram_range,max_features=max_features)
X_train=vectorizer.fit_transform(X_train)
X_test=vectorizer.transform(X_test)


In [10]:
# Apply SMOTE to handle inbalanced data
smote=SMOTE(random_state=42)
X_train,y_train=smote.fit_resample(X_train,y_train)



In [11]:
# Function to log results in MLflow
def log_mlflow(model_name, model, X_train, X_test, y_train, y_test, params, trial_number):
    with mlflow.start_run():
        # Log model type and trial number
        mlflow.set_tag("mlflow.runName", f"Trial_{trial_number}_{model_name}_SMOTE_TFIDF_Trigrams")
        mlflow.set_tag("experiment_type", "algorithm_comparison")

        # Log algorithm name as a parameter
        mlflow.log_param("algo_name", model_name)

        # Log hyperparameters
        for key, value in params.items():
            mlflow.log_param(key, value)

        # Train model
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # Log accuracy
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        # Log classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Log the model
        mlflow.sklearn.log_model(model, f"{model_name}_model")

        return accuracy




In [12]:
# Step 6: Optuna objective function for LightGBM
def objective_lightgbm(trial):
    # Hyperparameter space to explore
    n_estimators = trial.suggest_int('n_estimators', 100, 1000)
    learning_rate = trial.suggest_float('learning_rate', 1e-4, 1e-1, log=True)
    max_depth = trial.suggest_int('max_depth', 3, 15)
    num_leaves = trial.suggest_int('num_leaves', 20, 150)
    min_child_samples = trial.suggest_int('min_child_samples', 10, 100)
    colsample_bytree = trial.suggest_float('colsample_bytree', 0.5, 1.0)
    subsample = trial.suggest_float('subsample', 0.5, 1.0)
    reg_alpha = trial.suggest_float('reg_alpha', 1e-4, 10.0, log=True)  # L1 regularization
    reg_lambda = trial.suggest_float('reg_lambda', 1e-4, 10.0, log=True)  # L2 regularization

    # Log trial parameters
    params = {
        'n_estimators': n_estimators,
        'learning_rate': learning_rate,
        'max_depth': max_depth,
        'num_leaves': num_leaves,
        'min_child_samples': min_child_samples,
        'colsample_bytree': colsample_bytree,
        'subsample': subsample,
        'reg_alpha': reg_alpha,
        'reg_lambda': reg_lambda
    }

    # Create LightGBM model
    model = LGBMClassifier(n_estimators=n_estimators,
                           learning_rate=learning_rate,
                           max_depth=max_depth,
                           num_leaves=num_leaves,
                           min_child_samples=min_child_samples,
                           colsample_bytree=colsample_bytree,
                           subsample=subsample,
                           reg_alpha=reg_alpha,
                           reg_lambda=reg_lambda,
                           random_state=42)

    # Log each trial as a separate run in MLflow
    accuracy = log_mlflow("LightGBM", model, X_train, X_test, y_train, y_test, params, trial.number)

    return accuracy




In [13]:
# Step 7: Run Optuna for LightGBM, log the best model, and plot the importance of each parameter
def run_optuna_experiment():
    study = optuna.create_study(direction="maximize")
    study.optimize(objective_lightgbm, n_trials=100)  # Increased to 100 trials

    # Get the best parameters
    best_params = study.best_params
    best_model = LGBMClassifier(n_estimators=best_params['n_estimators'],
                                learning_rate=best_params['learning_rate'],
                                max_depth=best_params['max_depth'],
                                num_leaves=best_params['num_leaves'],
                                min_child_samples=best_params['min_child_samples'],
                                colsample_bytree=best_params['colsample_bytree'],
                                subsample=best_params['subsample'],
                                reg_alpha=best_params['reg_alpha'],
                                reg_lambda=best_params['reg_lambda'],
                                random_state=42)

    # Log the best model with MLflow and print the classification report
    log_mlflow("LightGBM", best_model, X_train, X_test, y_train, y_test, best_params, "Best")

    # Plot parameter importance
    optuna.visualization.plot_param_importances(study).show()

    # Plot optimization history
    optuna.visualization.plot_optimization_history(study).show()

In [14]:
# Run the experiment for LightGBM
run_optuna_experiment()

[I 2025-03-08 03:03:03,996] A new study created in memory with name: no-name-8546ca5c-9ad8-4c4a-9e53-2ae94547065b
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Number of positive: 12616, number of negative: 12616
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.120613 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 57101
[LightGBM] [Info] Number of data points in the train set: 25232, number of used features: 693
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with p

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
2025/03/08 03:03:25 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
[I 2025-03-08 03:03:28,707] Trial 0 finished with value: 0.8265000879816998 and parameters: {'n_estimators': 407, 'learning_rate': 0.006637439113100485, 'max_depth': 14, 'num_leaves': 23, 'min_child_samples': 98, 'colsample_bytree': 0.8616128055513101, 'subsample': 0.6435849625319907, 'reg_alpha': 0.2214017224090471, 'reg_lambda': 0.41216201180141243}. Best is trial 0 with value: 0.8265000879816998.


🏃 View run Trial_0_LightGBM_SMOTE_TFIDF_Trigrams at: http://ec2-100-26-36-125.compute-1.amazonaws.com:5000/#/experiments/621100004064027216/runs/1616e6b222284ea7b5648c28a8ffec0a
🧪 View experiment at: http://ec2-100-26-36-125.compute-1.amazonaws.com:5000/#/experiments/621100004064027216


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


...

🏃 View run Trial_99_LightGBM_SMOTE_TFIDF_Trigrams at: http://ec2-100-26-36-125.compute-1.amazonaws.com:5000/#/experiments/621100004064027216/runs/436574d7924742a5b0ebfed2d6c12f34
🧪 View experiment at: http://ec2-100-26-36-125.compute-1.amazonaws.com:5000/#/experiments/621100004064027216


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Number of positive: 12616, number of negative: 12616
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 1.765001 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 111143
[LightGBM] [Info] Number of data points in the train set: 25232, number of used features: 5726
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further 

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
2025/03/08 04:03:54 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Trial_Best_LightGBM_SMOTE_TFIDF_Trigrams at: http://ec2-100-26-36-125.compute-1.amazonaws.com:5000/#/experiments/621100004064027216/runs/81ac30e030a04e0685dc2cdc5ed05358
🧪 View experiment at: http://ec2-100-26-36-125.compute-1.amazonaws.com:5000/#/experiments/621100004064027216
